## 챕터 7: 함수 데커레이터와 클로저

* 파이썬이 데커레이터 구문을 평가하는 방식
* 변수가 지역 변수인지 파이썬이 판단하는 방식
* 클로저의 존재 이유와 작동 방식
* nonlocal로 해결할 수 있는 문제

### 데커레이터 기본 지식
- 데커레이터는 편리 구문일 뿐이다. 데커레이터는 다른 함수를 인수로 전달해서 호출하는 일반적인 콜러블과 동일하다. 그렇지만 런타임에 프로그램 행위를 변경하는 **메타프로그래밍**을 할 때 데커레이터가 상당히 편리하다.

### 파이썬이 데커레이터를 실행하는 시점
- 데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬이 모듈을 로딩하는 시점, 즉 **임포트 타임**에 실행된다.

In [1]:
registry = [] #registry 배열은 @register로 데커레이트된 함수들에 대한 참조를 담는다.

def register(func): #register()는 함수를 인수로 받는다.
    print('running register(%s)' % func)
    registry.append(func)
    return func 

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main(): #main()은 registry를 출력하고 f1(), f2(), f3()를 차례로 호출한다.
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x7fb5f7ae3b90>)
running register(<function f2 at 0x7fb5f7ae3e60>)
running main()
registry -> [<function f1 at 0x7fb5f7ae3b90>, <function f2 at 0x7fb5f7ae3e60>]
running f1()
running f2()
running f3()


- 함수 데커레이터는 모듈이 임포트되자마자 실행되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있다.

### 데커레이터로 개선한 전략 패턴

In [2]:
# promotion 데커레이터로 채운 promos 리스트
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total()* .1
        return discount
    
@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distince_items) >= 10:
        return order.total()*.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)
    

- 장점:
    프로모션 전략 함수명이 특별한 형태로 있을 필요X
    @promotion 데커레이터는 데커레이트된 함수의 목적을 명확히 알려주며, 임시로 어떤 프로모션을 배제할 수 있다. 단지 데커레이트만 주석처리.
    프로모션 할인 전략을 구현한 함수를 @promotion 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다.
    
### 변수 함수 규칙
함수 안에 할당하는 문장이 있지만 인터프리터가 b를 전역 변수로 다루기 원한다면, 다음과 같이 global 키워드를 이용해서 선언해야 한다.

In [3]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [4]:
f3(3)

3
6


In [5]:
b

9

In [6]:
f3(3)

3
9


In [8]:
b = 30
b

30

### 클로저
- 클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonglobal) 변수를 포함한 확장 범위를 가진 함수이다. 함수가 익명 함수인지 여부는 중요하지 않다. 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다. 

### nonlocal 선언

In [10]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

### 간단한 데커레이터 구현하기

In [11]:
# 함수의 실행 시간을 출력하는 간단한 데커레이터
import time

def clock(func):
    def clocked(*args): # 내부 함수 clocked()가 임의 개수의 위치 인수를 받을 수 있도록 정의
        t0 = time.perf_counter()
        result = func(*args) # clocked()에 대한 클로저에 자유 변수 func가 들어가야 이 코드가 작동한
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' %(elapsed, name, arg_str, result))
        return result
    return clocked #내부 함수를 반환해서 데커레이트된 함수를 대체한다.

In [ ]:
# clock 데커레이터 사용하기
import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6!=', factorial(6))

#### 작동 과정
본질적으로 clocked()함수는 다음과 같은 연산을 수행
1. 초기 시각 t0을 기록한다.
2. 원래의 factorial()함수를 호출하고 결과를 저장한다.
3. 흘러간 시간을 계산한다.
4. 수집한 데이터를 포맷하고 출력한다.
5. 2번째 단계에서 저장한 결과를 반환한다.

### 표준 라이브러리에서 제공하는 데커레이터

#### functools.lru_cache()를 이용한 메모이제이션
- functools.lru_cache()는 메모이제이션을 구현한다. 메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다.

In [ ]:
import functools

from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ =='__main__':
    print(fibonacci(6))

#### 단일 디스패치를 이용한 범용 함수

In [17]:
# 여러 함수를 범용 함수로 묶는 커스텀htmlize.register()를 생성하는 singledispatch
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch #@singledispatch()는 객체형을 다룰 기반 함수를 표시한다.
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) #각각의 특화된 함수는 @<기반_함수>.register(<객체형>)으로 데커레이트된다.
def _(text): # 특화된 함수의 이름은 필요 없으므로 언더바로 함수명을 지정한다.
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) #특별하게 처리할 자료형을 추가할 때마다 새로운 함수를 등록한다. 
# numbers.Integral은 int의 가상 슈퍼클래스다.
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple) # 동일한 함수로 여러 자료형을 지원하기 위해 register 데커레이터를 여러 개 쌓아올릴 수 있다.
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

### 매개변수화된 데커레이터

#### 매개변수화된 등록 데커레이터

In [19]:
# 매개변수를 받기 위해 함수로 호출되어야 하는 새로운 register() 데커레이터
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' %(active, func))
        if active: #클로저에서 읽어온 active 인수가 True일 때만 funcfmf emdfhrgksek.
            registry.add(func)
        else:
            registry.discard(func) #active가 True가 아니고 func가 registry에 들어 있으면 제거한다.
        
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register() # 인수를 전달하지 않더라도 register는 여전히 함수로 호출해야 하므로 @register() 형태로 호출한다.
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7fb5f7badf80>)
running register(active=True)->decorate(<function f2 at 0x7fb5f7badef0>)


#### 매개변수화된 clock 데커레이터

In [22]:
# clockdeco_param.py 모듈: 매개변수화된 clock() 데커레이터
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): #clock은 매개변수화된 데커레이터 팩토리이다.
    def decorate(func): #decorate()은 실제 데커레이터다.
        def clocked(*_args): #clocked()은 데커레이트된 함수를 래핑한다.
            t0 = time.time()
            _result = func(*_args) #데커레이트된 함수의 실제 결과를 _result에 저장한다.
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) #_args가 실제 clocked()의 인수를 담고 있으며, args는 출력하기 위한 문자열이다.
            result = repr(_result) #result는 출력하기 위해 _result를 문자열로 표현한 것이다.
            print(fmt.format(**locals())) #여기서 **locals()를 사용하면 fmt가 clocked()의 지역 변수를 모두 참조할 수 있게 해준다.            
            return _result # clocked()는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환해야 한다.
        return clocked # decorate()는 clocked()를 반환한다.
    return decorate # clock()은 decorate()를 반환한다.

if __name__=='__main__':
    
    @clock() #이 테스트에서는 인수 없이 clock()을 호출하므로, 적용된 데커레이터는 기본 포맷 문자열을 사용한다.
    def snooze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        snooze(.123)

[0.12517881s] snooze(0.123) -> None
[0.12793493s] snooze(0.123) -> None
[0.12596512s] snooze(0.123) -> None


## 챕터 8: 객체 참조, 가변성, 재활용

### 변수는 상자가 아니다

In [23]:
# 객체가 생성된 후에야 변수가 객체에 할당된다.
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' %id(self))

In [24]:
x = Gizmo() 

Gizmo id: 140419521422224


In [25]:
y = Gizmo()*10

Gizmo id: 140419522093328


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

### 정체성, 동질성, 별명

**모든 객체는 정체성, 자료형, 값을 가지고 있다. 객체의 정체성은 일단 생성한 후에는 결코 변경되지 않는다. 정체성은 메모리 내의 객체 주소라고 생각할 수 있다. is 연산자는 두 객체의 정체성을 비교한다. id() 함수는 정체성을 나타내는 정수를 반환한다.**

#### ==연산자와 is 연산자 간의 선택
== 연산자(동치 연산자)가 **객체의 값**을 비교하는 반면, is 연산자는 **객체의 정체성**을 비교한다.

#### 튜플의 상대적 불변성
튜플의 불변성은 tuple 데이터 구조체의 물리적인 내용(즉, 참조 자체)만을 말하는 것이며, 참조된 객체까지 불변성을 가지는 것은 아니다.

### 기본 복사는 얕은 복사
생성자나 [:]을 사용하면 **얕은 사본**을 생성한다. 즉 최상위 컨테이너는 복제하지만 사본은 원래 컨테이너에 들어 있던 동일 객체에 대한 참조로 채워진다. 

#### 객체의 깊은 복사와 얕은 복사
내포된 객체의 참조를 공유하지 않도록 깊게 복사할 필요가 있는 경우가 종종 있다. copy모듈이 제공하는 deepcopy() 함수는 깊은 복사를, copy()함수는 얕은 복사를 지원한다.

In [29]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(slef, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [30]:
import copy
bus1 = Bus(['Alice', 'Bill', ' Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

### 참조로서의 함수 매개변수
파이썬은 **공유로 호출**하는 매개변수 전달 방식만 지원한다. 이 반식은 루비, 스콜토크, 자바 등 대부분의 객체지향 언어에서 사용하는 방식과 동일하다. 공유로 호출한다는 말은 함수의 각 매개변수가 인수로 전달받은 각 참조의 사본을 받는다는 의미이다.

이런 체계의 결과로서, 함수는 인수로 전달받은 모든 가변 객체를 변경할 수 있지만, 객체의 정체성 자체는 변경할 수 없다. 즉 어떤 객체를 다른 객체로 변경할 수 없다. 

#### 가변형을 매개변수 기본값으로 사용하기: 좋지 않은 생각

#### 가변 매개변수에 대한 방어적 프로그래밍

In [36]:
class TwilightBus:
    """승객이 사라지게 만드는 버스 모델"""
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            # passengers 리스트의 사본을 만들거나, passengers가 리스트가 아닐 때는 리스트로 변환한다.
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

### del과 가비지 컬렉션

In [37]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye(): #이 함수는 제거될 객체의 메서드에 바인딩되거나 제거될 객체를 참조하면 안 된다.
    print('Gone with the wind...')

In [39]:
ender = weakref.finalize(s1, bye) #s1이 가리키는 객체에 대해 bye()콜백을 등록한다.
ender.alive #finalize 객체가 호출되기 전의 alive속성은 참

True

In [40]:
del s1
ender.alive #앞에서 설명한대로 del은 객체가 아니라 객체에 대한 참조를 제거한다.

True

In [41]:
s2 = 'spam' #마지막 참조인 s2를 다른 객체에 바인딩하면 {1, 2, 3}튜플에 도달할 수 없게 된다.
#튜플이 제거되고, bye()콜백이 호출되고, ender.alive는 거짓이 된다.

Gone with the wind...
Gone with the wind...


In [42]:
ender.alive

False

### 약한 참조

객체가 메모리에 유지되거나 유지되지 않도록 만드는 것은 참조의 존재 여부다. 객체 참조 카운트가 0이 되면 가비지 컬렉터는 해당 객체를 제거한다. 그러나 불필요하게 객체를 유지시키지 않으면서 객체를 참조할 수 있으면 도움이 되는 경우가 종종 있다. 캐시가 대표적이다.

약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조한다. 참조의 대상인 객체를 **참조 대상**이라고 한다. 따라서 약한 참조는 참조 대상이 가비지 컬렉트가 되는 것을 방지하지 않는다고 말할 수 있다.

- 약한 참조는 콜러블이다. 객체가 살아있으면 참조된 객체를 반환하고, 그렇지 않으면 None을 반환한다.

In [43]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x7fb5f7b15050; to 'set' at 0x7fb5f7aee7d0>

In [44]:
wref()

{0, 1}

In [45]:
a_set = {2, 3, 4}
wref()

{0, 1}

In [46]:
wref() is None #표현식을 평가할 때 {0, 1}이 살아 있으므로 wref()는 None이 아니다. 
#그렇지만 _변수는 결과값인 False에 바인딩된다. 이제 _변수는 더 이상 {0, 1}을 참조하지 않는다.

False

In [47]:
wref() is None

False

#### WeakValueDictionary 촌극
WeakValueDictionary 클래스는 객체에 대한 약한 참조를 값으로 가지는 가변 매핑을 구현한다. 참조된 객체가 프로그램 다른 곳에서 가비지 컬렉트되면 해당 키도 WeakValueDictionary에서 자동으로 제거된다. 이 클래스는 일반적으로 캐시를 구현하기 위해 사용된다.